# Preprocessamento
---

Este notebook tem o objetivo de realizar o pré-processamento, para verificação de cada uma das _features_. Serão observados valores faltantes, tipos de dado e faixas de valores para as variáveis presentes.

No pipeline desenvolvido, este notebook é a segunda parte a ser executada, visto que a primeira foi o script de transformação do arquivo _'sales_data.tsv'_ para o formato parquet, localizado em `scripts/csv_to_parquet.py` na pasta raíz do projeto.

## Importação das bibliotecas necessárias

In [1]:
import pandas as pd
from datetime import datetime

## Uma primeira primeira impressão dos dados

Aqui é feita a leitura do arquivo parquet para verificar, inicialmente, como os dados se parecem.

In [2]:
df = pd.read_parquet('../data/raw/sales_data.parquet')

In [3]:
df.dtypes

purchase_id                          int64
product_id                         float64
affiliate_id                       float64
producer_id                        float64
buyer_id                           float64
purchase_date                       object
product_creation_date               object
product_category                    object
product_niche                       object
purchase_value                     float64
affiliate_commission_percentual    float64
purchase_device                     object
purchase_origin                     object
is_origin_page_social_network       object
dtype: object

In [4]:
df.head()

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network
0,0,NaN,NaN,NaN,NaN,None,None,None,None,NaN,NaN,None,None,0
1,1663958,6640.0,209372.0,116238.0,1200397.0,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,-0.265302,50.0,Smart TV,Origin ef2b,"0,0"
2,1677087,2350.0,141418.0,2821.0,1083764.0,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,-0.177077,60.0,Smart TV,Origin ef2b,"0,0"
3,2017360,35669.0,618642.0,618642.0,1436106.0,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.468989,0.0,Smart TV,Origin ef2b,"0,0"
4,2017379,57998.0,1164511.0,70388.0,1436118.0,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,-0.401168,50.0,Smart TV,Origin ef2b,"0,0"


É possível observar que o primeiro registro da tabela tem todos valores inválidos, com _0_ ou _NaN_. Por isso, o mesmo será removido.

In [5]:
df = df.drop(0, axis=0)

A princípio, os últimos registros estão em conformidade:

In [6]:
df.tail()

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network
1599824,14011995,238362.0,7586641.0,7586641.0,5736172.0,2016-06-30 23:59:57,2016-06-16 12:10:46,Phisical book,Personal finance,-0.345361,0.0,eReaders,Origin 3022,"0,0"
1599825,14012431,61279.0,589022.0,589022.0,946067.0,2016-06-30 21:40:11,2013-06-15 16:41:06,Phisical book,Personal finance,-0.471786,0.0,Smart TV,Origin ef2b,"0,0"
1599826,14343996,215242.0,1186145.0,1186145.0,6473172.0,2016-05-13 16:45:42,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,"0,0"
1599827,14344113,215242.0,1186145.0,1186145.0,6473172.0,2016-06-22 14:39:05,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,"0,0"
1599828,14357203,215242.0,1186145.0,1186145.0,6473172.0,2016-04-11 19:37:25,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Tablet,Origin 3fcc,"0,0"


## Variáveis de identificação (ids)

O primeiro conjunto de colunas a serem verificadas são as que possuem campos de identificação.

### Ajuste de tipo

  Como os dados foram salvos em diretamente de uma consulta SQL, é possível que algumas colunas não estejam com a tipagem correta. Esse é o caso de alguns dos campos de identificação (que estão em formato `int` ou `float`), por isso estes serão convertidos em _strings_, visto que servirão apenas para identificar registros e não serão submetidos a nenhum tipo de operação aritmética.

In [7]:
# selecionando as colunas com '_id'
id_columns = [col for col in df.columns if '_id' in col]

# visualização dos tipos de dados  encontrados
df[id_columns].dtypes

purchase_id       int64
product_id      float64
affiliate_id    float64
producer_id     float64
buyer_id        float64
dtype: object

In [8]:
# Como purchase_id está com tipo inteiro, basta converter para string
df.loc[:, 'purchase_id'] = df['purchase_id'].astype(str)
df['purchase_id'].head()

1    1663958
2    1677087
3    2017360
4    2017379
5    2017382
Name: purchase_id, dtype: object

In [9]:
# Remove da lista o purchase_id, que já foi corrigido
id_columns.remove('purchase_id')

In [10]:
# Para as outras colunas de identificadores, realiza o ajuste de tipo
for id_col in id_columns:
    
    # Primeiro transforma em inteiro, para remover a parte decimal
    df.loc[:, id_col] = df[id_col].astype(int)
    
    # Depois transforma em string
    df.loc[:, id_col] = df[id_col].astype(str)

In [11]:
# Adiciona o purchase_id, novamente para visualização
id_columns.append('purchase_id')

In [12]:
df[id_columns].head()

,product_id,affiliate_id,producer_id,buyer_id,purchase_id
1,6640,209372,116238,1200397,1663958
2,2350,141418,2821,1083764,1677087
3,35669,618642,618642,1436106,2017360
4,57998,1164511,70388,1436118,2017379
5,58329,1261488,221253,1386357,2017382


Dessa forma, todos campos que se referem a "id" ficam com o tipo correto `string` ou `object`, o equivalente no pandas.

In [13]:
df[id_columns].dtypes

product_id      object
affiliate_id    object
producer_id     object
buyer_id        object
purchase_id     object
dtype: object

### Checagem de valores faltantes

In [14]:
'''
Função que conta o número de valores faltantes,
dada uma coluna do DataFrame.
'''
def check_nan(column):
    
    total_nan = df[column].isna().sum()
    
    if total_nan == 0:
        perc_nan = 0
    else:
        perc_nan = df.shape[0] / total_nan
    
    
    print("Coluna:", column)
    print("Total valores inválidos:", total_nan)
    print("Percentual de valores inválidos:{:.1f}%\n".format(perc_nan*100))

Os campos identificadores não possuem nenhum valor faltante, como pode ser conferido abaixo:

In [15]:
for id_col in id_columns:
    check_nan(id_col)

Coluna: product_id
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: affiliate_id
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: producer_id
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: buyer_id
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: purchase_id
Total valores inválidos: 0
Percentual de valores inválidos:0.0%



## Variáveis de data

### Ajuste de tipo

Nesta etapa serão verificados aqueles dados que tem o tipo data.

Inicialmente, já podemos verificar que os dados estão com o tipo errado (`object`).

In [16]:
# selecionando as colunas com '_date'
date_columns = [col for col in df.columns if '_date' in col]

# visualização dos tipos de dados  encontrados
df[date_columns].dtypes

purchase_date            object
product_creation_date    object
dtype: object

Para converter para o tipo `datetime`, primeiro é preciso verificar o formato em que a data está organizada. O formato utilizado é `Y-m-d H:M:S`, portanto será definida a função `to_datetime()`para conversão usando esse formato.

In [17]:
df[date_columns].head()

,purchase_date,product_creation_date
1,2016-06-26 12:00:00,2011-03-19 15:47:36
2,2016-06-26 12:00:00,2010-07-05 01:50:15
3,2016-06-26 12:00:00,2012-06-13 02:59:37
4,2016-06-26 12:00:00,2013-05-07 08:51:31
5,2016-06-26 12:00:00,2013-05-12 08:12:06


In [18]:
def to_datetime(str_date):
    """
    Retorna str_date no formato datetime.

    Argumentos:
    str_date -- string no formato %Y-%m-%d %H:%M:%S a ser convertida
    """
    return datetime.strptime(str_date, '%Y-%m-%d %H:%M:%S')

Após essa definição, é necessário aplicar a função às colunas com data (`purchase_date` e `product_creation_date`)

In [19]:
# Para as colunas de datas, realiza o ajuste de tipo
for date_col in date_columns:
    
    # Transformação da coluna para o tipo data
    df.loc[:, date_col] = df[date_col].apply(to_datetime)

Verifica-se que o tipo está correto.

In [20]:
df[date_columns].head()

,purchase_date,product_creation_date
1,2016-06-26 12:00:00,2011-03-19 15:47:36
2,2016-06-26 12:00:00,2010-07-05 01:50:15
3,2016-06-26 12:00:00,2012-06-13 02:59:37
4,2016-06-26 12:00:00,2013-05-07 08:51:31
5,2016-06-26 12:00:00,2013-05-12 08:12:06


In [21]:
df[date_columns].dtypes

purchase_date            datetime64[ns]
product_creation_date    datetime64[ns]
dtype: object

### Checagem de valores faltantes

Todos os campos de data estão preenchidos com valores válidos.

In [22]:
for date_col in date_columns:
    check_nan(date_col)

Coluna: purchase_date
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: product_creation_date
Total valores inválidos: 0
Percentual de valores inválidos:0.0%



## Variáveis categóricas

Vamos considerar como variáveis categóricas aquelas que possuem um número finito de valores possíveis, que não servem como _id_ e que podem ser vistos como categorias ou classes.

In [23]:
cat_columns =  [
    'product_category',
    'product_niche',
    'purchase_device',
    'purchase_origin',
    'is_origin_page_social_network'
]

### Checagem de valores faltantes

Nenhuma das colunas possuem lacunas nos valores.

In [24]:
for cat_col in cat_columns:
    check_nan(cat_col)

Coluna: product_category
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: product_niche
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: purchase_device
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: purchase_origin
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: is_origin_page_social_network
Total valores inválidos: 0
Percentual de valores inválidos:0.0%



### Ajuste de tipo

A princípio, todas variáveis categóricas estão com o tipo correto.

In [25]:
df[cat_columns].dtypes

product_category                 object
product_niche                    object
purchase_device                  object
purchase_origin                  object
is_origin_page_social_network    object
dtype: object

Além disso, também é importante ver quais os valores possíveis para cada coluna.

In [26]:
for cat_col in cat_columns:
    print(cat_col)
    print(df[cat_col].unique())

product_category
['Video' 'Podcast' 'Phisical book' 'eBook' 'In-class course' 'Workshop'
 'Webinar' 'eTicket' 'Subscription' 'App']
product_niche
['Presentation skills' 'Child psychology' 'Anxiety management'
 'Teaching English' 'Online course creation' 'Media training'
 'Storytelling' 'YouTube video creation' 'Procrastination' 'Organization'
 'Negotiation' 'Careers' 'Personal finance' 'Filmmaking' 'Government'
 'Global diplomacy' 'Immigration' 'Economics' 'Accounting' 'Biology'
 'Physics' 'Genetics' 'Disease' 'Thermodynamics' 'Travel hacking']
purchase_device
['Smart TV' 'Tablet' 'Desktop' 'eReaders' 'Cellphone']
purchase_origin
['Origin ef2b' 'Origin 3fcc' 'Origin 5187' ... 'Origin 6530' 'Origin fe8e'
 'Origin 290d']
is_origin_page_social_network
['0,0' '1,0']


É possível observar que `is_origin_page_social_network` se trata de uma variável binária, então seus valores poderiam ser convertidos em _0_ ou _1_', invés de '0,0' eou '1,0'.

In [27]:
df.loc[:, 'is_origin_page_social_network'] = df['is_origin_page_social_network'].str.replace('0,0', '0')
df.loc[:, 'is_origin_page_social_network'] = df['is_origin_page_social_network'].str.replace('1,0', '1')
df.loc[:, 'is_origin_page_social_network'] = df['is_origin_page_social_network'].astype(int)
df['is_origin_page_social_network'].unique()

array([0, 1])

## Variáveis numéricas

As últimas variáveis a serem analisadas serão as duas variáveis numéricas, as quais dizem respeito ao valor da compra e o percentual de comissão do afiliado, `purchase_value` e `affiliate_commission_percentual`, respectivamente.

In [28]:
num_columns = ['purchase_value', 'affiliate_commission_percentual']

### Checagem de valores faltantes

Nenhum valor faltante.

In [29]:
for num_col in num_columns:
    check_nan(num_col)

Coluna: purchase_value
Total valores inválidos: 0
Percentual de valores inválidos:0.0%

Coluna: affiliate_commission_percentual
Total valores inválidos: 0
Percentual de valores inválidos:0.0%



### Ajuste de tipo

Ambas colunas estão com o tipo correto `float`.

In [30]:
df[num_columns].dtypes

purchase_value                     float64
affiliate_commission_percentual    float64
dtype: object

## Extensão de variáveis

Uma das ténicas de pré-processamento utilizada nas áreas de Aprendizado de Máquina e Ciência de Dados é criar novas _features_ a partir do conjunto de dados original. Portanto, nesta etapa serão adicionadas algumas colunas para facilitar futuros processamentos.

### Compra com afiliado

Criação de uma nova coluna para verificar mais rapidamente se a compra foi feita por intermédio de um afiliado ou não.  Assim, a nova coluna `is_affliate_purchase` será `1` quando a compra tiver sido feita por meio de afiliado e `0` caso contrário.

In [31]:
# Pega os índices em que o id do produtor é igual ao do afiliado e o percentual de comissão é zero
affiliate_purchases_indexes = df[(df['producer_id'] == df['affiliate_id']) &
                                 (df['affiliate_commission_percentual'] == 0)].index

In [32]:
# Cria uma coluna, inicialmente, com toda compra setada com afiliado
df['is_affliate_purchase'] = 1

# Atualiza as compras sem afiliados, com zero
df.loc[affiliate_purchases_indexes, 'is_affliate_purchase'] = 0

In [33]:
df

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,is_affliate_purchase
1,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,-0.265302,50.0,Smart TV,Origin ef2b,0,1
2,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,-0.177077,60.0,Smart TV,Origin ef2b,0,1
3,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.468989,0.0,Smart TV,Origin ef2b,0,0
4,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,-0.401168,50.0,Smart TV,Origin ef2b,0,1
5,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,-0.452489,50.0,Smart TV,Origin ef2b,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599824,14011995,238362,7586641,7586641,5736172,2016-06-30 23:59:57,2016-06-16 12:10:46,Phisical book,Personal finance,-0.345361,0.0,eReaders,Origin 3022,0,0
1599825,14012431,61279,589022,589022,946067,2016-06-30 21:40:11,2013-06-15 16:41:06,Phisical book,Personal finance,-0.471786,0.0,Smart TV,Origin ef2b,0,0
1599826,14343996,215242,1186145,1186145,6473172,2016-05-13 16:45:42,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,0,0
1599827,14344113,215242,1186145,1186145,6473172,2016-06-22 14:39:05,2016-03-26 17:59:47,Phisical book,Negotiation,-0.359158,0.0,Smart TV,Origin ef2b,0,0


### Valor de compra fictício

O z-score pode ser utilizado em diversas ocasiões, como em aplicações de Aprendizado de Máquina, onde, muitas vezes, a standardização é desejável. Contudo, como o valor da compra, na coluna `purchase_value`, está 'escondido' no z-score, outras análises sobre esse conjunto de dados podem ser prejudicadas. Por isso, para critérios de demonstração, vamos assumir uma média e um desvio padrão para, a partir do z-score fornecido, criar valores fictícios de compra.

Para garantir que uma média e desvio padrão válidos sejam escolhidos, é necessário ver os valores máximos e mínimos do z-score para evitar gerar valores negativos (o que não seria possível de acontecer em uma compra).

In [34]:
# Setando visualização com 2 casas decimais
pd.options.display.float_format = '{:,.2f}'.format

In [35]:
df['purchase_value'].describe()

count   1,599,828.00
mean            0.00
std             1.00
min            -0.54
25%            -0.45
50%            -0.35
75%             0.06
max           124.56
Name: purchase_value, dtype: float64

Sendo $z$ definido pela equação:

$$ z = \frac{x - \mu}{\sigma} $$

Onde $x$ é o valor original, $\mu$ é a média e $\sigma$ o desvio padrão. A fórmula reversa do z-score seria:

$$ x = z + \sigma \times \mu $$

Sendo assim, vamos assumir uma média e desvio padrão fictício para criar os dados fictícios de valor de compra.

In [36]:
def reverse_zscore(z, mean, std):
    '''
    Reverte o zscore para o valor original,
    dada a média e desvio padrão.
    
    Argumentos:
    
      z -- Pandas Series com os z-scores
      mean -- média
      std -- desvio padrão
    
    '''
    x = z * std + mean
    return x

Escolha de uma média e desvio padrão arbitrários, mas que não geram valores negativos para os valores de compra.

In [37]:
# média
new_mean = 100
# desvio padrão
new_std = 150

Aplicação da função que 'reverte' o z-score.

In [38]:
df['new_purchase_value'] = reverse_zscore(df['purchase_value'], new_mean, new_std)

Nenhum valor fora da faixa de valores válidos.

In [39]:
df['new_purchase_value'].describe()

count   1,599,828.00
mean          100.00
std           150.00
min            18.81
25%            32.05
50%            47.50
75%           109.74
max        18,784.15
Name: new_purchase_value, dtype: float64

In [40]:
df

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,is_affliate_purchase,new_purchase_value
1,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,-0.27,50.00,Smart TV,Origin ef2b,0,1,60.20
2,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,-0.18,60.00,Smart TV,Origin ef2b,0,1,73.44
3,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.47,0.00,Smart TV,Origin ef2b,0,0,29.65
4,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,-0.40,50.00,Smart TV,Origin ef2b,0,1,39.82
5,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,-0.45,50.00,Smart TV,Origin ef2b,0,1,32.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599824,14011995,238362,7586641,7586641,5736172,2016-06-30 23:59:57,2016-06-16 12:10:46,Phisical book,Personal finance,-0.35,0.00,eReaders,Origin 3022,0,0,48.20
1599825,14012431,61279,589022,589022,946067,2016-06-30 21:40:11,2013-06-15 16:41:06,Phisical book,Personal finance,-0.47,0.00,Smart TV,Origin ef2b,0,0,29.23
1599826,14343996,215242,1186145,1186145,6473172,2016-05-13 16:45:42,2016-03-26 17:59:47,Phisical book,Negotiation,-0.36,0.00,Smart TV,Origin ef2b,0,0,46.13
1599827,14344113,215242,1186145,1186145,6473172,2016-06-22 14:39:05,2016-03-26 17:59:47,Phisical book,Negotiation,-0.36,0.00,Smart TV,Origin ef2b,0,0,46.13


## Salvamento do _data set_

Após essa etapa de pré-processamento, o conjunto de dados pode  ser salvo novamente.

In [41]:
df.to_parquet('../data/preprocessed/sales_data.parquet')

Conferindo se o parquet ficou com os tipos corretos.

In [42]:
df = pd.read_parquet('../data/preprocessed/sales_data.parquet')
df

,purchase_id,product_id,affiliate_id,producer_id,buyer_id,purchase_date,product_creation_date,product_category,product_niche,purchase_value,affiliate_commission_percentual,purchase_device,purchase_origin,is_origin_page_social_network,is_affliate_purchase,new_purchase_value
1,1663958,6640,209372,116238,1200397,2016-06-26 12:00:00,2011-03-19 15:47:36,Video,Presentation skills,-0.27,50.00,Smart TV,Origin ef2b,0,1,60.20
2,1677087,2350,141418,2821,1083764,2016-06-26 12:00:00,2010-07-05 01:50:15,Podcast,Child psychology,-0.18,60.00,Smart TV,Origin ef2b,0,1,73.44
3,2017360,35669,618642,618642,1436106,2016-06-26 12:00:00,2012-06-13 02:59:37,Podcast,Presentation skills,-0.47,0.00,Smart TV,Origin ef2b,0,0,29.65
4,2017379,57998,1164511,70388,1436118,2016-06-26 12:00:00,2013-05-07 08:51:31,Podcast,Anxiety management,-0.40,50.00,Smart TV,Origin ef2b,0,1,39.82
5,2017382,58329,1261488,221253,1386357,2016-06-26 12:00:00,2013-05-12 08:12:06,Podcast,Teaching English,-0.45,50.00,Smart TV,Origin ef2b,0,1,32.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599824,14011995,238362,7586641,7586641,5736172,2016-06-30 23:59:57,2016-06-16 12:10:46,Phisical book,Personal finance,-0.35,0.00,eReaders,Origin 3022,0,0,48.20
1599825,14012431,61279,589022,589022,946067,2016-06-30 21:40:11,2013-06-15 16:41:06,Phisical book,Personal finance,-0.47,0.00,Smart TV,Origin ef2b,0,0,29.23
1599826,14343996,215242,1186145,1186145,6473172,2016-05-13 16:45:42,2016-03-26 17:59:47,Phisical book,Negotiation,-0.36,0.00,Smart TV,Origin ef2b,0,0,46.13
1599827,14344113,215242,1186145,1186145,6473172,2016-06-22 14:39:05,2016-03-26 17:59:47,Phisical book,Negotiation,-0.36,0.00,Smart TV,Origin ef2b,0,0,46.13


In [43]:
df.dtypes

purchase_id                                object
product_id                                 object
affiliate_id                               object
producer_id                                object
buyer_id                                   object
purchase_date                      datetime64[ns]
product_creation_date              datetime64[ns]
product_category                           object
product_niche                              object
purchase_value                            float64
affiliate_commission_percentual           float64
purchase_device                            object
purchase_origin                            object
is_origin_page_social_network               int64
is_affliate_purchase                        int64
new_purchase_value                        float64
dtype: object